## **Momento 1**

#### Importar o Dataset.

O objetivo é usar o algoritmo K-means para agrupar bairros de diferentes cidades brasileiras com base em duas variáveis principais: *a quantidade de área verde disponível* (Ha) e *a densidade populacional* (m²). 

## **Momento 2**

#### Instalar e carregar as bibliotecas que serão usadas para leitura de dados, aplicação do K-means e visulização dos resultados.

In [ ]:
library(readxl) # permite importar dados diretamente de planilhas do Excel
library(dplyr) # permite transformar dados
library(cluster) # fornece funções para análise de cluster
library(factoextra) # ajuda a criar gráficos que tornam os resultados da análise mais nítidos
library(ggplot2) # permite criar gráficos complexos e personalizados de maneira intuitiva

## **Momento 3**

#### Acessar e fazer a leitura do Dataset.

Importe os dados de acordo com o formato do arquivo que você possui.

In [ ]:
# Especificar o caminho do arquivo correto
file_path <- ('/kaggle/input/mydata/MyDataFrame.xlsx')

# Carregar o arquivo xlsx em um dataframe
Dados_Originais <- read_excel('/kaggle/input/mydata/MyDataFrame.xlsx', sheet = 'Plan1')

# Exibir as 7 primeiras linhas do dataframe
head(Dados_Originais, 7)

## **Momento 4**
#### Filtre o Dataset e selecione os dados a serem utilizados.
Se você possui um Dataset com multiplas dimensões e não pretende utilizar todas elas, selecione aquelas que você deseja utilizar.

In [ ]:
# Selecione as variavis de interesse
Dados_Filtrados <- Dados_Originais[,c("Green.Spaces.Area", "Population.Density")]
head(Dados_Filtrados, 7)

## **Momento 5**
#### Coheça os dados
Você pode plotar desde um gráfico mais geral, para entender a relação dos dados, até um gráfico mais específico, para ver a distribuição dos dados.

In [ ]:
# Grafico para visualizar a relação entre os dados
plot(Dados_Filtrados)

In [ ]:
# Boxplot para Green Spaces Area
Chart_Green_Spaces_Area<- boxplot(Dados_Filtrados$Green.Spaces.Area, 
        main = "Boxplot de Green Spaces Area", 
        ylab = "Green Spaces Area",
        col = "lightgreen", 
        border = "black")

# Boxplot para Population Density
Population_Density<- boxplot(Dados_Filtrados$Population.Density, 
        main = "Boxplot de Population Density", 
        ylab = "Population Density",
        col = "pink", 
        border = "black")

## **Momento 6**
#### Prepare os dados
Uma boa prática é normalizar os dados.
 Isso é especialmente importante se as unidades de medida forem muito diferentes (por exemplo, metros quadrados para áreas verdes e pessoas para população). Normalizar os dados coloca todas as variáveis na mesma escala, evitando que uma variável domine as outras na análise.

In [ ]:
# Visualizando os dados a serem utilizados
head(Dados_Filtrados, 7)

In [ ]:
# Normalizando os dados
Dados_Normalizados <- Dados_Filtrados %>%
  select(Green.Spaces.Area, Population.Density) %>%
  scale()  # Normalizar os dados


# read:
head(Dados_Normalizados, 7)

In [ ]:
# Boxplot para Green Spaces Area e Population Density
Chart_Green_Spaces_Area<- boxplot(Dados_Normalizados, 
        main = "Boxplot de Green Spaces Area", 
        ylab = "Values",
        col = "darkgreen", 
        border = "black")

## **Momento 7**

#### Escolha do número de Clusters (K).

Para determinar o número ideal de clusters vamos, aplicar o método do cotovelo.

In [ ]:
# Determinar o número ideal de clusters usando o método do cotovelo 
fviz_nbclust(Dados_Normalizados, kmeans, method = "wss") +
  geom_vline(xintercept = 3, linetype = 2, color = "red") +  
  annotate("text", x = 4.1, y = 205, label = "Best number of K", color = "red", vjust = -0.9)  

## **Momento 8**

#### Aplicação do K-means.

Aplicar o algoritmo K-means aos dados normalizados. O algoritmo dividirá os bairros em k clusters baseados nas semelhanças entre a Área Verde e a Densidade Populacional.

In [ ]:
# Aplicar o algoritmo K-means
set.seed(123)  # Para garantir a reprodutibilidade
kmeans_result <- kmeans(Dados_Normalizados, centers = 3) # Aplicar o algoritmo K-means considerando K=3

## **Momento 9**

#### Visualização e interpretação dos resultados.

Após a execução do K-means, cada bairro será atribuído a um dos clusters. Analisar cada cluster para entender suas características médias pode revelar padrões.

In [ ]:
clusterchart<- fviz_cluster(kmeans_result, data = Dados_Filtrados, label=NA)+
  theme_bw()
clusterchart

## **Momento 10**

#### Integrar os clusters ao DataFrame original.
Aqui é possível integrar os resultados ao DataFrame inicial.

In [ ]:
# Adicionar o resultado do clustering aos dados originais
Dados_Originais$Cluster <- kmeans_result$cluster

# Ler o novo DataFrame
head(Dados_Originais)

## **Momento 11**

#### Salavar as imagens.
Aqui é possível salvar as imagens para posteriormente baixar, caso queira.

In [ ]:
# Definir o caminho e o tamanho da imagem
file_path <- "/kaggle/working/meu_grafico_ggplot.png"
width <- 10
height <- 7

# Salvar o gráfico com as dimensões especificadas
ggsave(filename = file_path, plot = clusterchart, width = width, height = height, units = "in", dpi = 300)

# Verificar se o arquivo foi salvo corretamente
list.files("/kaggle/working/")

## **Momento 12**

#### Apagar as imagens salvas.
Aqui é possível limpar o Output.

In [ ]:
# Remover todos os arquivos do diretório de trabalho
unlink("/kaggle/working/*", recursive = TRUE)

# Verificar se o arquivo foi salvo corretamente
list.files("/kaggle/working/")

## **Momento 13**

#### Criar criar novos gráficos com o Dataframe.
Aqui é possível criar novos gráficos e realizar novas análises.

In [ ]:
# Crie o gráfico de dispersão com ggplot2
ggplot(Dados_Originais, aes(x = Green.Spaces.Area, y = Population.Density, color = as.factor(Cluster))) +
  geom_point(size = 4) + # Tamanho dos pontos
  labs(
    title = "Dispersão de Área de Espaços Verdes e Densidade Populacional",
    x = "Área de Espaços Verdes",
    y = "Densidade Populacional",
    color = "Cluster"
  ) +
  theme_bw() + # Tema do gráfico
  scale_color_manual(values = c("blue", "red", "green")) # Definindo as cores para os clusters

In [ ]:
# Boxplot para a densidade populacional por cluster
ggplot(Dados_Originais, aes(x = as.factor(Cluster), y = Population.Density, fill = as.factor(Cluster))) +
  geom_boxplot() +
  labs(
    title = "Distribuição da Densidade Populacional por Cluster",
    x = "Cluster",
    y = "Densidade Populacional",
    fill = "Cluster"
  ) +
  scale_fill_manual(values = c("darkgray", "darkblue", "pink")) + # Definindo as cores para os clusters
theme_bw()

# Você pode responder perguntas como: 


#### 1- Quais bairros têm características semelhantes em termos de área verde e densidade populacional? 
K-means agrupa os bairros em clusters onde bairros dentro do mesmo cluster têm valores semelhantes para área verde e densidade populacional. Você poderá identificar quais bairros possuem características semelhantes e, portanto, podem ter necessidades ou perfis comparáveis em termos de espaços verdes e densidade populacional.


#### 2- Há padrões de distribuição de áreas verdes em relação à densidade populacional? 
Ao analisar os clusters, você pode identificar como a quantidade de área verde se relaciona com a densidade populacional em diferentes bairros. Com essas novas informações, é possível analisar se bairros com alta densidade populacional tendem a ter menos área verde ou se há clusters onde a área verde é proporcional à densidade populacional.


#### 3- Como diferentes bairros se apresentam em termos de acesso a áreas verdes?
Observando os clusters, é possível identificar quais bairros dentro de um cluster têm mais ou menos área verde disponível. Isso pode ajudar a comparar o acesso a áreas verdes entre diferentes clusters e a identificar quais grupos de bairros possuem melhor ou pior acesso a áreas verdes em relação à sua densidade populacional.

# Situações onde você eventualmente usará K-means

#### Clasificação de propriedades agrícolas:

Demanda: A empresa deseja entender as características de diferentes propriedades agrícolas para fornecer soluções personalizadas.

Uso do K-means: Agrupar propriedades com base em variáveis como: tamanho da propriedade e área plantada. Isso permite oferecer recomendações específicas para fertilização, irrigação e manejo de culturas.


#### Análise de serviços ecossistêmicos fornecidos:

Situação: Avaliar serviços ecossistêmicos, sequestro de carbono e controle de erosão do solo.

Uso do K-means: Agrupar áreas com base na capacidade de sequestro de carbono e controle de erosão. Isso pode ajudar a priorizar áreas para conservação com base na importância dos serviços que elas prestam.


#### Planejamento municipal para restauração e conservação de eocssitemas de Veredas degradadas:

Desafio: Planejar e priorizar ações de restauração e conservação de eocssitemas de Veredas degradadas em municípios do Vale do Araguaia.

Uso do K-means: Agrupar municipios veredas base em critérios de necessidade de restauração, como área (km²) degradadas e e custo potencial de recuperação. Isso ajuda a direcionar os esforços de restauração para municípios em situação crítica.

# Exercício

## Faça a leitura dos dados da 'Plan2' existente no Dataset

In [ ]:
# Especificar o caminho do arquivo correto
file_path <- ('/kaggle/input/mydata/MyDataFrame.xlsx')

# Carregar o arquivo xlsx em um dataframe
Dados_Originais <- read_excel('/kaggle/input/mydata/MyDataFrame.xlsx', sheet = 'Plan2')

# Exibir as 7 primeiras linhas do dataframe
head(Dados_Originais, 7)

## Selecione as varáveis de interesse

In [ ]:
# Selecione as variavis de interesse
Meus_Dados_Selecionados <- Dados_Originais[,c("Renda", "Densidade")]
head(Dados_Filtrados, 7)

## Normalize os dados

In [ ]:
# Normalizando os dados
Meus_Dados_Normalizados <- Meus_Dados_Selecionados %>%
  select(Renda, Densidade) %>%
  scale()  # Normalizar os dados


# read:
head(Dados_Normalizados, 7)

## Plote os dados normalizados

In [ ]:
# Boxplot para Green Spaces Area e Population Density
Rena_verus_Densidade_Populacional<- boxplot(Meus_Dados_Normalizados, 
        main = "Boxplot de Green Spaces Area", 
        ylab = "Values",
        col = "darkgreen", 
        border = "black")

## Encontre o valor ótimo de K pelo método do colotovelo

In [ ]:
# Determinar o número ideal de clusters usando o método do cotovelo 
fviz_nbclust(Meus_Dados_Normalizados, kmeans, method = "wss") +
  geom_vline(xintercept = 3, linetype = 2, color = "red") +  
  annotate("text", x = 4.1, y = 205, label = "Best number of K", color = "red", vjust = -0.9)  

# Aplique o algoritmo K-means aos dados

In [ ]:
# Aplicar o algoritmo K-means
set.seed(123)  # Para garantir a reprodutibilidade
kmeans_result_meus_dados <- kmeans(Meus_Dados_Normalizados, centers = 3) # Aplicar o algoritmo K-means considerando K=3

# Plote o resultado da K-means 

In [ ]:
clusterchart<- fviz_cluster(kmeans_result_meus_dados, data = Meus_Dados_Normalizados, label=NA)+
  theme_bw()
clusterchart